In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [4]:
def read_driver_data(filename):
    data = pd.read_csv(filename, delim_whitespace=True, header=None, 
            names=['Year', 'DOY', 'hr', 'min', 'Bx', 'By', 'Bz', 'Vx', 'Vy', 'Vz', 'Np', 'T', 'SYM-H', 'IMF flag', 'SW flag', 'Tilt', 'Pdyn', 'N-index', 'B-index'])
    data['ID'] = data['Year']*10000000 + data['DOY']*10000 + data['hr']*100 + data['min']
    return data

def read_omni_data(filename):
    data = pd.read_csv(filename, delim_whitespace=True, header=None, 
            names=['Year', 'DOY', 'hr', 'B','V', 'phi', 'theta', 'Kp'])
    data['ID'] = data['Year']*100000 + data['DOY']*100 + data['hr']*1
    return data

def compute_N(Vx, Vy, Vz, By, Bz):
    V = np.sqrt(Vx**2 + Vy**2 + Vz**2)
    Bt = np.sqrt(By**2  + Bz**2)
    th = np.arctan2(By, Bz)
    N = 0.86*(V/400)**(4/3) * (Bt/5)**(2/3) * (np.sin(th/2) ** 8) ** (1/3)
    return N.values[0]

def compute_B(Vx, Vy, Vz, By, Bz, Np):
    V = np.sqrt(Vx**2 + Vy**2 + Vz**2)
    Bt = np.sqrt(By**2  + Bz**2)
    th = np.arctan2(By, Bz)
    B = (Np/5)**0.5 * (V / 400)**2.5 * Bt / 5 * np.sin(th/2) ** 6
    return B.values[0]


def input_varying_Vx(driverfilename, omnifilename, timestamp, samplesize):
    data = read_driver_data(driverfilename)
    omni = read_omni_data(omnifilename)
    selectOmni = omni[omni.ID == round(timestamp/100)]
    select = data[data.ID == timestamp]
    
    lists = []
    for i in range(samplesize):
        Vz = 0
        Vy = 0
        Vx = random.uniform(-350,-450)
        
        B = compute_B(Vx, Vy, Vz, select.By, select.Bz, select.Np)

        V = np.sqrt(Vx**2 + Vy**2 + Vz**2)
        Pdyn = select.Np.values[0]/1000000*V**2 * 2

        nv = {'VGSEX' : Vx, 'VGSEY' : Vy, 'VGSEZ' : Vz, 
              'PDYN' : Pdyn, 'B0y': select.By.values[0], 
              'B0z': select.Bz.values[0], 'XIND': B, 
              'PS': select.Tilt.values[0], 'IOPT': int(selectOmni.Kp.values[0]/10)
             }

        lists.append(nv)

    df = pd.DataFrame(lists)
    df.to_csv('inputfiles/input_Vx')
    return df


def input_varying_B(driverfilename, omnifilename, timestamp, samplesize):
    data = read_driver_data(driverfilename)
    omni = read_omni_data(omnifilename)
    selectOmni = omni[omni.ID == round(timestamp/100)]
    select = data[data.ID == timestamp]
    
    lists = []
    for i in range(samplesize):
        scale = random.uniform(0.75,1.25)
        Bx = select.Bx*scale
        By = select.By*scale
        Bz = select.Bz*scale
        
        Vx = select.Vx
        Vy = select.Vy
        Vz = select.Vz
        
        
        B = compute_B(Vx, Vy, Vz, By, Bz, select.Np)

        V = np.sqrt(Vx.values[0]**2 + Vy.values[0]**2 + Vz.values[0]**2)
        Pdyn = select.Np.values[0]/1000000*V**2 * 2

        nv = {'VGSEX' : Vx.values[0], 'VGSEY' : Vy.values[0], 'VGSEZ' : Vz.values[0], 
              'PDYN' : Pdyn, 'B0y': By.values[0], 
              'B0z': Bz.values[0], 'XIND': B, 
              'PS': select.Tilt.values[0], 'IOPT': int(selectOmni.Kp.values[0]/10)
             }

        lists.append(nv)

    df = pd.DataFrame(lists)
    df.to_csv('inputfiles/input_B')
    return df

input_varying_B('data/2004_OMNI_5m_with_TA15_drivers.dat', 'data/omni_20040509_T89.txt', 20041290900, 30)
input_varying_Vx('data/2004_OMNI_5m_with_TA15_drivers.dat', 'data/omni_20040509_T89.txt', 20041290900, 30)

,VGSEX,VGSEY,VGSEZ,PDYN,B0y,B0z,XIND,PS,IOPT
0,-385.008150,0,0,1.058371,-1.95,-3.18,0.455322,0.2768,2
1,-380.107791,0,0,1.031601,-1.95,-3.18,0.440972,0.2768,2
2,-436.291671,0,0,1.359102,-1.95,-3.18,0.622424,0.2768,2
3,-412.195985,0,0,1.213125,-1.95,-3.18,0.540012,0.2768,2
4,-367.343466,0,0,0.963480,-1.95,-3.18,0.404878,0.2768,2
5,-366.042522,0,0,0.956668,-1.95,-3.18,0.401303,0.2768,2
6,-377.981883,0,0,1.020094,-1.95,-3.18,0.434832,0.2768,2
7,-443.270585,0,0,1.402930,-1.95,-3.18,0.647614,0.2768,2
8,-418.336469,0,0,1.249539,-1.95,-3.18,0.560348,0.2768,2
9,-426.143938,0,0,1.296614,-1.95,-3.18,0.586860,0.2768,2


In [34]:
print(compute_N(select.Vx, select.Vy, select.Vz, select.By, select.Bz))
print(compute_B(select.Vx, select.Vy, select.Vz, select.By, select.Bz, select.Np))

0.7937861419041128
0.752971457272638


In [14]:
select
0.86*(np.sqrt(select.Vx**2 + select.Vy**2 + select.Vz**2) / 400)**(4/3) * (np.sqrt(select.By**2  + select.Bz**2)/5)**(2/3) * np.sin(np.arctan(select.By/select.Bz)/2)**(8/3)

35766    0.027194
dtype: float64

In [10]:
data = pd.read_csv('omni_20040509_T89.txt', delim_whitespace=True, header=None, 
            names=['Year', 'DOY', 'hr', 'B','V', 'phi', 'theta', 'Kp'])
data['ID'] = data['Year']*100000 + data['DOY']*100 + data['hr']*1
select = data[data.ID == 200412909]

select

,Year,DOY,hr,B,V,phi,theta,Kp,ID
9,2004,129,9,4.6,483.0,-0.3,-3.1,20,200412909


In [12]:
print(round(1.7))
print(round(2.3))

2
2
